## Sentimate Analysis
* https://www.digitalocean.com/community/tutorials/how-to-perform-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk#step-1-%E2%80%94-installing-nltk-and-downloading-the-data

### Imports and Downloads

In [104]:
# Download Training Data
from nltk.corpus import twitter_samples

# used to filter tokens for meaningless words
from nltk.corpus import stopwords

# helps access each token's relative position of a word in a sentence
from nltk.tag import pos_tag

# helps group different forms of words into one token (different conjugations words, etc)
from nltk.stem.wordnet import WordNetLemmatizer

# helps determine the most common words
from nltk import FreqDist

# to split dataset for model
import random

### Tokenizing The Data
* Definition: processing language to make it easier for the machine to understand
* How: splits strings into smaller parts called tokens 

In [105]:
# viewing files inside twitter_samples data
twitter_samples.fileids()

['negative_tweets.json', 'positive_tweets.json', 'tweets.20150430-223406.json']

In [106]:
# converting all tweets in dataset to strings for easier processing
positive_tweets = twitter_samples.strings("positive_tweets.json")
negative_tweets = twitter_samples.strings("negative_tweets.json")
random_tweets = twitter_samples.strings("tweets.20150430-223406.json")
# tokenizing tweets
pos_tweet_tokens = twitter_samples.tokenized("positive_tweets.json")
neg_tweet_tokens = twitter_samples.tokenized("negative_tweets.json")
random_tweet_tokens = twitter_samples.tokenized("tweets.20150430-223406.json")
pos_tweet_tokens[0]

['#FollowFriday',
 '@France_Inte',
 '@PKuchly57',
 '@Milipol_Paris',
 'for',
 'being',
 'top',
 'engaged',
 'members',
 'in',
 'my',
 'community',
 'this',
 'week',
 ':)']

### Normalizing The Data
* Converting differnent forms of words into the same form (Ex: run, runs, running)
* Stemming: removing affixes from a word, removes ends of words
* Lemmatization: analyzes the structure of the word and its context to convert it to a normalized form

In [107]:
# tag tokens with proper positional tags
positive_tagged = pos_tag(pos_tweet_tokens[0])
negitive_tagged = pos_tag(neg_tweet_tokens[0])
random_tagged = pos_tag(random_tweet_tokens[0])
positive_tagged

[('#FollowFriday', 'JJ'),
 ('@France_Inte', 'NNP'),
 ('@PKuchly57', 'NNP'),
 ('@Milipol_Paris', 'NNP'),
 ('for', 'IN'),
 ('being', 'VBG'),
 ('top', 'JJ'),
 ('engaged', 'VBN'),
 ('members', 'NNS'),
 ('in', 'IN'),
 ('my', 'PRP$'),
 ('community', 'NN'),
 ('this', 'DT'),
 ('week', 'NN'),
 (':)', 'NN')]

In [108]:
# # Lemmatizing Sentence example (not necessary, done in next block)
# # Normalizes tweet with lemmatize_sentence (Ex: being=be, members=member)
# def lemmatize_sentence(tokens):
#     lemmatizer = WordNetLemmatizer()
#     lemmatized_sentence = []
#     for word, tag in pos_tag(tokens):
#         if tag.startswith('NN'):
#             pos = 'n'
#         elif tag.startswith('VB'):
#             pos = 'v'
#         else:
#             pos = 'a'
#         lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
#     return lemmatized_sentence

# print(lemmatize_sentence(pos_tweet_tokens[0]))

In [109]:
# removes hyperlinks, twitter handles, puncuation/special characters from single tweet
import re, string

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

# words converted to lower case, puncuation and links removed. Text is also lemmatized
stop_words = stopwords.words('english')
cleaned_tokens = remove_noise(pos_tweet_tokens[0], stop_words)
print(f"Original Tweet: {pos_tweet_tokens[0]}")
print(f"Example: {cleaned_tokens}")

Original Tweet: ['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'being', 'top', 'engaged', 'members', 'in', 'my', 'community', 'this', 'week', ':)']
Example: ['#followfriday', 'top', 'engage', 'member', 'community', 'week', ':)']


In [110]:
# running on all pos/neg tokens
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in pos_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in neg_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

In [111]:
# printing example of all tweets in dataset tokenized
print(positive_tweet_tokens[500])
print(positive_cleaned_tokens_list[500])

['Dang', 'that', 'is', 'some', 'rad', '@AbzuGame', '#fanart', '!', ':D', 'https://t.co/bI8k8tb9ht']
['dang', 'rad', '#fanart', ':d']


### Determining Word Density

In [112]:
# compile all cleaned words into one large distribution for analysis
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

all_pos_words = get_all_words(positive_cleaned_tokens_list)

# find the most common words
freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


### Preparing Data For Model
* Model associates tweets with a positive or negative sentiment

In [113]:
# coverts tweets from list to dictionary with keys as tokens and true as values
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

# splitting the dataset for traning and testing
positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]

In [114]:
# Building and Testing The Model
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))

Accuracy is: 0.9953333333333333
Most Informative Features
                      :( = True           Negati : Positi =   2064.5 : 1.0
                      :) = True           Positi : Negati =   1662.8 : 1.0
                     sad = True           Negati : Positi =     67.8 : 1.0
                follower = True           Positi : Negati =     32.9 : 1.0
                     bam = True           Positi : Negati =     23.6 : 1.0
                    sick = True           Negati : Positi =     21.7 : 1.0
                    glad = True           Positi : Negati =     21.6 : 1.0
              appreciate = True           Positi : Negati =     15.6 : 1.0
               community = True           Positi : Negati =     15.0 : 1.0
                 welcome = True           Positi : Negati =     14.4 : 1.0
None


### Testing On Custom Sentence

In [115]:
# Testing on custom tweet
from nltk.tokenize import word_tokenize

custom_tweet = "Mib is good"

custom_tokens = remove_noise(word_tokenize(custom_tweet))

print(classifier.classify(dict([token, True] for token in custom_tokens)))

Positive


### Improvements
* Cannot detect sarcasm
* Cannot detect "Hi" vs "Hiiiii"